<a href="https://colab.research.google.com/github/chunyuan0221/3rd-ML100days/blob/master/Day_100_transfer_learning_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
from keras.datasets import cifar10
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from google.colab import files
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

Using TensorFlow backend.


In [3]:
f = files.upload()

Saving resnet_builder.py to resnet_builder.py


In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [5]:
# 讀取資料集並作前處理
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [6]:
from resnet_builder import resnet
model = resnet(input_shape=(32,32,3)) 
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [8]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
generator = ImageDataGenerator(rotation_range=10,
                                    width_shift_range=0.15,
                                    height_shift_range=0.15,
                                    zoom_range=0.15,
                                    shear_range=0.15,
                                    horizontal_flip=True)

xtrain_gen = generator.flow(x_train, y_train, batch_size=batch_size)
history = model.fit_generator(xtrain_gen,
                              steps_per_epoch=len(x_train) // batch_size,
                              epochs=epochs,
                              verbose=1,
                              validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/30
390/390 [==============================] - 38s 98ms/step - loss: 1.4615 - acc: 0.6182 - val_loss: 2.0010 - val_acc: 0.5002
Epoch 2/30
390/390 [==============================] - 33s 84ms/step - loss: 1.3039 - acc: 0.6626 - val_loss: 1.8553 - val_acc: 0.5283
Epoch 3/30
390/390 [==============================] - 33s 83ms/step - loss: 1.1853 - acc: 0.6982 - val_loss: 1.5323 - val_acc: 0.6320
Epoch 4/30
390/390 [==============================] - 33s 83ms/step - loss: 1.0974 - acc: 0.7286 - val_loss: 1.6229 - val_acc: 0.6009
Epoch 5/30
390/390 [==============================] - 33s 83ms/step - loss: 1.0271 - acc: 0.7484 - val_loss: 1.4243 - val_acc: 0.6417
Epoch 6/30
390/390 [==============================] - 32s 83ms/step - loss: 0.9778 - acc: 0.7634 - val_loss: 1.1925 - val_acc: 0.7092
Epoch 7/30
390/390 [==============================] - 32s 83ms/step - loss: 0.9301 - acc: 0.7757 - val_loss: 1.2666 - val_acc: 0.6966
Epoch 8/30
390/390 [==============================] - 32s 83ms

### Result
***
- 本次的code運用到Transfer learning，以及ImageGenerator。
  - 透過Transfer learning我們可以將大神預訓好模型直接帶入使用，減少我們訓練的時間。
  - 加入ImageGenerator幫助我們強化訓練資料的數據量，避免overfitting。

- 我們觀察訓練過程，發現在某幾個epoch驗證集上的準確率有不錯的表現，我們也可加入callback的技巧，留住準確率高的epoch。